Import necessary libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import csv


Get that webpage and do some basic actions with it

In [9]:
# idea for web scraping: scrape audible best-selling books
URL = "https://www.audible.com/adblbestsellers?ref=a_hp_c6_adblp13nbssxbestsellers-d_showmore&pf_rd_p=26b86f23-d7fa-4f7e-bd31-f714f6e5625a&pf_rd_r=M14NV35SJTCTS8F4E3WT"
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.content, "html.parser")

In [14]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html>
<head>
<link href="//images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href=" //m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://images-na.ssl-images-amazon.com" rel="preconnect"/>
<link href=" https://m.media-amazon.com" rel="preconnect"/>
<meta content="width=1040" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/audiblemobile/store/image/favicons/icons16px.png" rel="icon" sizes="16x16" type="image/png">
<link href="https://images-na.ssl-images-amazon.com/images/G/01/audiblemobile/store/image/favicons/icons32px.png" rel="icon" sizes="32x32" type="image/png">
<link href="https://m.media-amazon.com/images/G/01/Audible/images/navigation/MobileWeb_IOSandAndroid_460._SY192_CB651482445_.png" rel="icon" sizes="192x192" type="image/png">
<link href="/favicon.ico" rel="shortcut icon"/>
<link href="https://m.media-amazon.com/images/G/01/Audible/images/navigation/MobileWeb_IOSandAndroid_